# Introduction:



### Lending Club enables borrowers to create unsecured personal loans. The standard loan period is three years. Investors can search and browse the loan listings on Lending Club website and select loans that they want to invest in based on the information supplied about the borrower, amount of loan, loan grade, and loan purpose. Investors make money from interest. Lending Club makes money by charging borrowers an origination fee and investors a service fee.


#### In this project one has to put themself in the shoes of a loan issuer and manage credit risk by using the past data and deciding whom to give the loan to in the future. The text files contain complete loan data for all loans issued by XYZ Corp. through 2007-2015. The data contains the indicator of default, payment information, credit history, etc.

# Loan Default Prediction

##### Project by <b>Pushkar Raj </b>

# Loading the text data

    
    

In [1]:
# Setting the path to my local directory where my dataset is placed

import os
os.chdir('E:\\Finale Project\\Python Project')
print (os.getcwd())

E:\Finale Project\Python Project


In [2]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing,metrics 
%matplotlib inline
from IPython.display import Image
from matplotlib import rcParams
rcParams['figure.figsize'] = 11,6


In [3]:
# Loading the data from txt file
data = pd.read_csv('XYZCorp_LendingData.txt',sep="\t",low_memory=False)
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855969 entries, 0 to 855968
Data columns (total 73 columns):
id                             855969 non-null int64
member_id                      855969 non-null int64
loan_amnt                      855969 non-null float64
funded_amnt                    855969 non-null float64
funded_amnt_inv                855969 non-null float64
term                           855969 non-null object
int_rate                       855969 non-null float64
installment                    855969 non-null float64
grade                          855969 non-null object
sub_grade                      855969 non-null object
emp_title                      806526 non-null object
emp_length                     812908 non-null object
home_ownership                 855969 non-null object
annual_inc                     855969 non-null float64
verification_status            855969 non-null object
issue_d                        855969 non-null object
pymnt_plan           

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,default_ind
count,8.559690e+05,8.559690e+05,855969.000000,855969.000000,855969.000000,855969.000000,855969.000000,8.559690e+05,855969.000000,855969.000000,...,11609.000000,13288.000000,13288.000000,13288.000000,13288.000000,7.886560e+05,13288.000000,13288.000000,13288.000000,855969.000000
mean,3.224073e+07,3.476269e+07,14745.571335,14732.378305,14700.061226,13.192320,436.238072,7.507119e+04,18.122165,0.311621,...,71.486993,1.354305,2.945515,5840.443332,61.024526,3.216357e+04,0.947772,1.524232,1.841963,0.054286
std,2.271969e+07,2.399418e+07,8425.340005,8419.471653,8425.805478,4.368365,243.726876,6.426447e+04,17.423629,0.857189,...,23.015293,1.483710,2.595313,5108.500262,20.018117,3.769964e+04,1.441667,2.697601,2.975049,0.226581
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,5.320000,15.690000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000,0.000000
25%,9.067986e+06,1.079273e+07,8000.000000,8000.000000,8000.000000,9.990000,260.550000,4.500000e+04,11.880000,0.000000,...,58.500000,0.000000,1.000000,2405.000000,47.900000,1.400000e+04,0.000000,0.000000,0.000000,0.000000
50%,3.431355e+07,3.697532e+07,13000.000000,13000.000000,13000.000000,12.990000,382.550000,6.500000e+04,17.610000,0.000000,...,75.000000,1.000000,2.000000,4485.500000,62.100000,2.380000e+04,0.000000,0.000000,2.000000,0.000000
75%,5.446311e+07,5.803559e+07,20000.000000,20000.000000,20000.000000,15.990000,571.560000,9.000000e+04,23.900000,0.000000,...,87.500000,2.000000,4.000000,7701.250000,75.300000,3.990000e+04,1.000000,2.000000,3.000000,0.000000
max,6.861687e+07,7.351969e+07,35000.000000,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,9999.000000,39.000000,...,223.300000,22.000000,43.000000,83047.000000,151.400000,9.999999e+06,15.000000,33.000000,32.000000,1.000000


In [ ]:
# Plot Loan Status
plt.figure(figsize= (12,6))
plt.ylabel('Loan Status')
plt.xlabel('Count')
data['default_ind'].value_counts().plot(kind = 'barh', grid = True)
plt.show()

In [ ]:
# Analyzing Purpose
total = len(data)

plt.figure(figsize = (14,6))

g = sns.countplot(x="", data=data, 
                  color='blue')
g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_xlabel("Loan Status Categories", fontsize=12)
g.set_ylabel("Count", fontsize=15)
g.set_title("Loan Status Types Distribution", fontsize=20)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=12) 
g.set_ylim(0, max(sizes) * 1.10)

plt.show()

In [ ]:
data.default_ind.value_counts()

In [ ]:
# Data Dimension
data.shape

In [ ]:
# Drop these features for now
data.drop([    'id',
             'member_id',
             'emp_title',
             'title',
#              'url',
             'zip_code',
             'verification_status',
             'home_ownership',
             'issue_d',
             'earliest_cr_line',
             'last_pymnt_d',
             'next_pymnt_d',
             'desc',
#             'pymnt_plan',
#             'initial_list_status',
#             'addr_state',
             'last_credit_pull_d', 
                                    ], axis=1, inplace=True)

In [ ]:
data.drop(['verification_status_joint'], axis=1, inplace=True)

In [ ]:
# Show records number
data.count().sort_values()

In [ ]:
# Drop columns with less than 25% data.
lack_of_data_idx = [x for x in data.count() < 855969*0.25]
data.drop(data.columns[lack_of_data_idx], 1, inplace=True)

In [ ]:
# After Deletion
data.info()

In [ ]:
print (data.mths_since_last_delinq.min(), data.mths_since_last_delinq.max())
print(data.mths_since_last_delinq.mean())
print(data.mths_since_last_delinq.median())

In [ ]:
data.mths_since_last_delinq = data.mths_since_last_delinq.fillna(data.mths_since_last_delinq.median())

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
# Calculate Good and Bad Loan Status Ratio
good_loan =  len(data[(data.default_ind == 0)])
print ('Good/Bad Loan Ratio: %.2f%%'  % (good_loan/len(data)*100))

In [ ]:
# create an bad/good loan indicator feature
data['good_loan'] = np.where((data.default_ind == 0) , 1, 0)

In [ ]:
# Hot encode some categorical features 
columns = ['term', 'grade', 'sub_grade', 'emp_length', 'purpose', 'application_type','addr_state',
           'pymnt_plan', 'initial_list_status']

for col in columns:
    tmp_df = pd.get_dummies(data[col], prefix=col)
    data = pd.concat((data, tmp_df), axis=1)

In [ ]:
# drop attributes that we hot-encoded
data.drop([#'loan_status',
           'term',
           'grade',
           'sub_grade',
           'emp_length',
           'addr_state',
           'initial_list_status',
           'pymnt_plan',
           'purpose',
           'application_type'], axis=1, inplace=True)

In [ ]:
# drop attributes that we hot-encoded
data.drop(['default_ind'], axis=1, inplace=True)

In [ ]:
# Rename some features to concur w/ some algorithms
data = data.rename(columns= {'emp_length_< 1 year':'emp_length_lt_1 year',
                         'emp_length_n/a':'emp_length_na'})

In [ ]:
# Due to resource limitation, we limit data to only the first 10,000 records.
data = data[:10000]

In [ ]:
data.head()

In [ ]:
# Split Train/Test data
from sklearn.model_selection import train_test_split

y = data['good_loan']
X = data.ix[:, data.columns != 'good_loan']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2, random_state=44)

In [ ]:
!pip install sklearn

In [ ]:
# Bring in evaluator
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score

# Flatten Data
from sklearn.preprocessing import StandardScaler, RobustScaler

#std_scaler = StandardScaler()
rob_scaler = RobustScaler()

#X_train_S = std_scaler.fit_transform(X_train)
#X_test_S = std_scaler.transform(X_test)

# Use robust scaler to reduce outliers
X_train_R = rob_scaler.fit_transform(X_train)
X_test_R = rob_scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Weighted prediction feature
y_0 = len(y_train[y_train == 0])/len(y_train)
y_1 = 1 - y_0

In [ ]:
svm_clf = SVC(class_weight={0:y_1, 1:y_0})
svm_clf.fit(X_train_R, y_train)

svm_predictions = svm_clf.predict(X_test_R) # Save prediction


#print(svm_clf.score(X_test_R, y_test))
scores = cross_val_score(svm_clf, X_test_R, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, svm_predictions))
print(mt.confusion_matrix(y_test, svm_predictions))

In [ ]:
!pip install joblib

In [ ]:
# SVM - SMOTE

from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=10, random_state=44, kind = 'svm')
X_res_train, y_res_train = sm.fit_sample(X_train_R, y_train)

In [ ]:
svm_sm_clf = SVC()
svm_sm_clf.fit(X_res_train, y_res_train)

svm_sm_predictions = svm_clf.predict(X_test_R)

#print(svm_sm_clf.score(X_test_R, y_test))
scores = cross_val_score(svm_sm_clf, X_test_R, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, svm_sm_predictions))
print(mt.confusion_matrix(y_test, svm_sm_predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 20)
rf.fit(X_train, y_train)
       
rf_predictions = rf.predict(X_test)

#print(rf.score(X_test, y_test))
scores = cross_val_score(rf, X_test, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, rf_predictions))
print(mt.confusion_matrix(y_test, rf_predictions))

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_predictions = xgb.predict(X_test)
                            
#print(xgb.score(X_test, y_test))
scores = cross_val_score(xgb, X_test, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, xgb_predictions))
print(mt.confusion_matrix(y_test, xgb_predictions))

# Another way(Re-run it again)

In [ ]:
# Taking the backup of loaded dataset
dataset_backup=data.copy()
dataset_backup.shape

#### Pre-processing Data- It appears there are a lot of NaNs and unnecessary data.

In [ ]:
missingdata = [x for x in data.count() < len(data)*0.35]
data.drop(data.columns[missingdata], axis=1, inplace=True)
data.shape
data.columns
len(data.columns)

In [ ]:
# Checking and dropping columns with just one unique value
unique = data.nunique()
unique = unique[unique.values == 1]
data.drop(labels = list(unique.index), axis =1, inplace=True)

In [ ]:
# Checking for duplicates
data.duplicated().value_counts()

In [ ]:
len(data.columns)

In [ ]:
data['annual_inc'].describe()

In [ ]:
# Checking for outliers Data types for numeric columns
data['annual_inc']= data['annual_inc'].astype(float)
data['annual_inc'].describe()
data_fin=data.drop(data[data.annual_inc>1e+05].index)

In [ ]:
# Bucketing and dropping extra columns

In [ ]:
data_fin['int_rate']=data_fin['int_rate'].astype(str)
data_fin['int_rate']= data_fin['int_rate'].map(lambda x: x.rstrip('%'))
data_fin['int_rate']= data_fin['int_rate'].astype(float)
data_fin['int_rate'].describe() 
buck = [0, 5, 10, 15, 20,25, 35]
lab = ['0-5', '5-10', '10-15', '15-20', '20-25','>25']
data_fin['int_rate_range'] = pd.cut(data_fin['int_rate'], buck, labels=lab)


In [ ]:
data_fin['loan_amnt'].describe() #0-40k
buck = [0, 5000, 10000, 15000, 20000, 25000,40000]
lab = ['0-5000', '5000-10000', '10000-15000', '15000-20000', '20000-25000','25000 and above']
data_fin['loan_amnt_range'] = pd.cut(data_fin['loan_amnt'], buck, labels=lab)

In [ ]:
data_fin['annual_inc'].describe() #range 1 to 1 mill
buck = [0, 25000, 50000, 75000, 100000,1000000]
lab = ['0-25000', '25000-50000', '50000-75000', '75000-100000', '100000 and above']
data_fin['annual_inc_range'] = pd.cut(data_fin['annual_inc'], buck, labels=lab)

Some basic plots to understand variable distribution- Univariate and bivariate relationships

In [ ]:
sns.distplot(data_fin['loan_amnt'])

In [ ]:
sns.distplot(data_fin['int_rate'])

In [ ]:
sns.distplot(data_fin['annual_inc'])

In [ ]:
sns.countplot(data_fin['default_ind'])
data_fin['default_ind'].value_counts()

In [ ]:
data_fin['default_ind'].value_counts()/len(data_fin)

In [ ]:
sns.countplot(data_fin['default_ind'])
sns.countplot(data_fin['purpose'],hue=data_fin['default_ind'])
sns.countplot(data_fin['purpose'],hue=data_fin['loan_amnt_range'])

In [ ]:
data_fin.drop(data_fin[data_fin['home_ownership']== 'ANY'].index, inplace=True)
data_fin.drop(data_fin[data_fin['home_ownership']== 'NONE'].index, inplace=True)
sns.countplot(data_fin['home_ownership'],hue=data_fin['default_ind'])

In [ ]:
#employment length
sns.countplot(data_fin['emp_length'],hue=data_fin['default_ind'])
sns.countplot(data_fin['emp_length'],hue=data_fin['loan_amnt_range'])

In [ ]:
#geography
sns.countplot(data_fin['addr_state'],hue=data_fin['default_ind']) 
sns.countplot(data_fin['addr_state'],hue=data_fin['loan_amnt_range'])

In [ ]:
# monthly trend
data_fin['issue_yr']=pd.DatetimeIndex(data_fin['issue_d']).year
data_fin['issue_mon']=pd.DatetimeIndex(data_fin['issue_d']).month
sns.countplot(data_fin['issue_mon'],hue=data_fin['default_ind'])

In [ ]:
cor_loan=data_fin[['loan_amnt','annual_inc', 'default_ind', 'int_rate', 'dti', 
               'tot_cur_bal', 'funded_amnt']]
f, ax = plt.subplots(figsize=(16, 9))
sns.heatmap(cor_loan.corr(), 
            xticklabels=cor_loan.columns.values,
            yticklabels=cor_loan.columns.values,annot= True)
plt.show()

In [ ]:
data_fin.drop(['id', 'member_id','zip_code' ,'emp_title' ,'funded_amnt', 'funded_amnt_inv', 'total_pymnt', 
               'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'out_prncp','out_prncp_inv', 
               'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'collections_12_mths_ex_med',
               'recoveries', 'collection_recovery_fee','title','revol_util'], axis=1, inplace=True)

In [ ]:
# Dropping some more unnecessary features
data_fin.drop(['next_pymnt_d', 'tot_coll_amt' , 'tot_cur_bal' ,'total_rev_hi_lim'], axis=1, inplace=True)


In [ ]:
len(data_fin.columns)

In [ ]:
data_fin['emp_length'].value_counts()
emp_range= {'< 1 year':0.5, '1 year':1, '2 years': 2, '3 years':3,
            '4 years':4, '5 years':5,'6 years':6,'7 years':7,
            '8 years':8,'9 years':9, '10+ years':10}
data_fin['emplen'] = data_fin["emp_length"].map(emp_range)
data_fin['emplen'].isnull().sum() 
data_fin['emplen'].value_counts()

### Categorizing data into categorical, continuous and count data.

In [ ]:
#Checking missing values in all the columns - how many null in each column
data_fin.isna()
data_fin.isnull().sum()

In [ ]:
nullseries=pd.isnull(data_fin).sum()
nullseries[nullseries>0]

data_fin['emplen'] = data_fin['emplen'].replace(np.nan, 10)
data_fin.drop(['emp_length'],axis=1,inplace=True)

data_fin['mths_since_last_delinq'] = data_fin['mths_since_last_delinq'].fillna(data_fin['mths_since_last_delinq'].median()) #mean and median v similar

#very few missing values for all of them
#data_fin['dti'] = data_fin['dti'].fillna(data_fin['dti'].mean())
#data_fin['inq_last_6mths'] = data_fin['inq_last_6mths'].fillna(data_fin['inq_last_6mths'].mean())
#data_fin['collections_12_mths_ex_med'] = data_fin['collections_12_mths_ex_med'].fillna(data_fin['collections_12_mths_ex_med'].mean())

# A lot of NAs- with integer values so median replacement
# NONE

In [ ]:
data_fin.isna()
data_fin.isnull().sum()

In [ ]:
verification_map={'Source Verified':3, 'Verified':2, 'Not Verified':1}
data_fin['verification_status']=data_fin['verification_status'].map(verification_map)

ownership_map={'MORTGAGE':1, 'RENT':2, 'OWN':3, 'OTHER':4}
data_fin['home_ownership']=data_fin['home_ownership'].map(ownership_map)

subgrade_map={'A1':1,'A2':2, 'A3':3, 'A4':4, 'A5':5, 'B1':6, 'B2':7, 'B3':8, 'B4':9, 'B5':10, 
              'C1':11, 'C2':12, 'C3':13, 'C4':14, 'C5':15, 'D1':16, 'D2':17, 'D3':18, 
              'D4':19, 'D5':20, 'E1':21, 'E2':22, 'E3':23, 'E4':24, 'E5':25, 'F1':26, 
              'F2':27, 'F3':28, 'F4':29, 'F5':30, 'G1':31, 'G2':32, 'G3':33, 'G4':34, 'G5':35}
data_fin['sub_grade']=data_fin['sub_grade'].map(subgrade_map)

In [ ]:
data_fin= data_fin[data_fin['purpose'] != 'educational']
data_fin= data_fin[data_fin['purpose'] !='wedding']
data_fin= data_fin[data_fin['purpose'] !='other']



In [ ]:
enc1= pd.get_dummies(data_fin['purpose'])
data_fin=pd.concat((data_fin,enc1), axis=1)
data_fin.drop(['purpose'],axis=1,inplace=True)

enc2= pd.get_dummies(data_fin['addr_state'])
data_fin=pd.concat((data_fin,enc2), axis=1)
data_fin.drop(['addr_state'],axis=1,inplace=True)

enc4= pd.get_dummies(data_fin['application_type'])
data_fin=pd.concat((data_fin,enc4), axis=1)
data_fin.drop(['application_type'],axis=1,inplace=True)

enc5= pd.get_dummies(data_fin['term'])
data_fin=pd.concat((data_fin,enc5), axis=1)
data_fin.drop(['term'],axis=1,inplace=True)

In [ ]:
data_fin.columns

In [ ]:
data_fin.drop(['earliest_cr_line','loan_amnt_range', 'annual_inc_range','int_rate_range','grade',
               'issue_yr','issue_mon','pymnt_plan','initial_list_status', 'total_rec_late_fee'],axis=1, inplace=True)
data_fin.isna()
data_fin.isnull().sum()


In [ ]:
len(data_fin.columns)

# # Univariate Analysis

In [ ]:
# emp_length
emp_length_count=data_fin['emplen'].value_counts()
print(emp_length_count)
bp=sns.barplot(emp_length_count.index,emp_length_count.values)
bp.set_xticklabels(bp.get_xticklabels(), rotation=45)

In [ ]:
# home_ownership
home_ownership_count=data_fin['home_ownership'].value_counts()
print(home_ownership_count)
sns.barplot(home_ownership_count.index,home_ownership_count.values)

In [ ]:
# acc_now_delinq
acc_now_delinq_count=data_fin['acc_now_delinq'].value_counts()
print(acc_now_delinq_count)
sns.barplot(acc_now_delinq_count.index,acc_now_delinq_count.values)

In [ ]:
# Since only one class dominates so we drop this feature
data_fin.drop(['acc_now_delinq'],axis=1,inplace=True)

In [ ]:
# annual income
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['annual_inc']);

plt.subplot(122)
data_fin['annual_inc'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# dti
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['dti']);

plt.subplot(122)
data_fin['dti'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "dti"
dti_mean = int(data_fin['dti'].mean())

#FInd 75th Percentile of the column "dti"
IQR_dti_P75 = data_fin['dti'].quantile(q=0.75)

#FInd 25th Percentile of the column "dti"
IQR_dti_P25 = data_fin['dti'].quantile(q=0.25)

#FInd IQR of the column "dti"
IQR_dti = IQR_dti_P75-IQR_dti_P25

#Fix boundaries to detect outliers in column "dti"
IQR_LL = int(IQR_dti_P25 - 1.5*IQR_dti)
IQR_UL = int(IQR_dti_P75 + 1.5*IQR_dti)

#treating upper end outier with mean
data_fin.loc[data_fin['dti']>IQR_UL , 'dti'] = dti_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['dti']<IQR_LL , 'dti'] = dti_mean

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['delinq_2yrs']);

plt.subplot(122)
data_fin['delinq_2yrs'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "delinq_2yrs"
delinq_2yrs_mean = int(data_fin['delinq_2yrs'].mean())

#FInd 75th Percentile of the column "delinq_2yrs"
IQR_delinq_2yrs_P75 = data_fin['delinq_2yrs'].quantile(q=0.75)

#FInd 25th Percentile of the column "delinq_2yrs"
IQR_delinq_2yrs_P25 = data_fin['delinq_2yrs'].quantile(q=0.25)

#FInd IQR of the column "dti"
IQR_delinq_2yrs = IQR_delinq_2yrs_P75-IQR_delinq_2yrs_P25

#Fix boundaries to detect outliers in column "dti"
IQR_LL = int(IQR_delinq_2yrs_P25 - 1.5*IQR_delinq_2yrs)
IQR_UL = int(IQR_delinq_2yrs_P75 + 1.5*IQR_delinq_2yrs)

#treating upper end outier with mean
data_fin.loc[data_fin['delinq_2yrs']>IQR_UL , 'delinq_2yrs'] = dti_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['delinq_2yrs']<IQR_LL , 'delinq_2yrs'] = dti_mean



In [ ]:
# open_acc
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['open_acc']);

plt.subplot(122)
data_fin['open_acc'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "open_acc"
open_acc_mean = int(data_fin['open_acc'].mean())

#FInd 75th Percentile of the column "open_acc"
IQR_open_acc_P75 = data_fin['open_acc'].quantile(q=0.75)

#FInd 25th Percentile of the column "open_acc"
IQR_open_acc_P25 = data_fin['open_acc'].quantile(q=0.25)

#FInd IQR of the column "open_acc"
IQR_open_acc = IQR_open_acc_P75-IQR_open_acc_P25

#Fix boundaries to detect outliers in column "open_acc"
IQR_LL = int(IQR_open_acc_P25 - 1.5*IQR_open_acc)
IQR_UL = int(IQR_open_acc_P75 + 1.5*IQR_open_acc)

#treating upper end outier with mean
data_fin.loc[data_fin['open_acc']>IQR_UL , 'open_acc'] = open_acc_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['open_acc']<IQR_LL , 'open_acc'] = open_acc_mean

In [ ]:
# pub_rec
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['pub_rec']);

plt.subplot(122)
data_fin['pub_rec'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "pub_rec"
pub_rec_mean = int(data_fin['pub_rec'].mean())

#FInd 75th Percentile of the column "pub_rec"
IQR_pub_rec_P75 = data_fin['pub_rec'].quantile(q=0.75)

#FInd 25th Percentile of the column "pub_rec"
IQR_pub_rec_P25 = data_fin['pub_rec'].quantile(q=0.25)

#FInd IQR of the column "pub_rec"
IQR_pub_rec = IQR_pub_rec_P75-IQR_pub_rec_P25

#Fix boundaries to detect outliers in column "pub_rec"
IQR_LL = int(IQR_pub_rec_P25 - 1.5*IQR_pub_rec)
IQR_UL = int(IQR_pub_rec_P75 + 1.5*IQR_pub_rec)

#treating upper end outier with mean
data_fin.loc[data_fin['pub_rec']>IQR_UL , 'pub_rec'] = pub_rec_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['pub_rec']<IQR_LL , 'pub_rec'] = pub_rec_mean

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['revol_bal']);

plt.subplot(122)
data_fin['revol_bal'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "revol_bal"
revol_bal_mean = int(data_fin['revol_bal'].mean())

#FInd 75th Percentile of the column "revol_bal"
IQR_revol_bal_P75 = data_fin['revol_bal'].quantile(q=0.75)

#FInd 25th Percentile of the column "revol_bal"
IQR_revol_bal_P25 = data_fin['revol_bal'].quantile(q=0.25)

#FInd IQR of the column "revol_bal"
IQR_revol_bal = IQR_revol_bal_P75-IQR_revol_bal_P25

#Fix boundaries to detect outliers in column "revol_bal"
IQR_LL = int(IQR_revol_bal_P25 - 1.5*IQR_revol_bal)
IQR_UL = int(IQR_revol_bal_P75 + 1.5*IQR_revol_bal)

#treating upper end outier with mean
data_fin.loc[data_fin['revol_bal']>IQR_UL , 'revol_bal'] = revol_bal_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['revol_bal']<IQR_LL , 'revol_bal'] = revol_bal_mean


In [ ]:
# total_acc
plt.figure(1)
plt.subplot(121)
sns.distplot(data_fin['total_acc']);

plt.subplot(122)
data_fin['total_acc'].plot.box(figsize=(16,5))

plt.show()

In [ ]:
# Outliers Treatment
#Find mean of the column "total_acc"
total_acc_mean = int(data_fin['total_acc'].mean())

#FInd 75th Percentile of the column "total_acc"
IQR_total_acc_P75 = data_fin['total_acc'].quantile(q=0.75)

#FInd 25th Percentile of the column "total_acc"
IQR_total_acc_P25 = data_fin['total_acc'].quantile(q=0.25)

#FInd IQR of the column "total_acc"
IQR_total_acc = IQR_total_acc_P75-IQR_total_acc_P25

#Fix boundaries to detect outliers in column "total_acc"
IQR_LL = int(IQR_total_acc_P25 - 1.5*IQR_total_acc)
IQR_UL = int(IQR_total_acc_P75 + 1.5*IQR_total_acc)

#treating upper end outier with mean
data_fin.loc[data_fin['total_acc']>IQR_UL , 'total_acc'] = total_acc_mean

#treating lower end outlier as mean
data_fin.loc[data_fin['total_acc']<IQR_LL , 'total_acc'] = total_acc_mean

## Checking Multicolinearity using Heatmap

In [ ]:
data_fin.columns.all

In [ ]:
# Categorizing Numeric and count data

data_numeric=data_fin.loc[:,['loan_amnt','int_rate','sub_grade', 'home_ownership','verification_status','inq_last_6mths',
       'annual_inc', 'dti', 'delinq_2yrs','open_acc','mths_since_last_delinq',  
       'total_acc', 'emplen', 'revol_bal',
       ]]

In [ ]:
#calculating correlation among numeric variable 
corr_matrix = data_numeric.corr() 

#plot correlation matrix
plt.figure(figsize=(20,12))
sns.heatmap(corr_matrix,
            cmap='coolwarm',
            annot=True);

In [ ]:
data_fin['issue_d'] = pd.to_datetime(data_fin['issue_d'])
data_fin = data_fin.set_index(data_fin['issue_d'])
data_fin = data_fin.sort_index()

train = data_fin['June 2007':'May 2015']
test  = data_fin['June 2015':'Dec 2015']

In [ ]:
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)

In [ ]:
train =train.drop('issue_d' , axis=1)
test =test.drop('issue_d', axis=1)

In [ ]:
from collections import Counter
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold

In [ ]:
from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTE 
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
X_train = train.drop('default_ind', axis=1)
y_train = train.loc[:,['default_ind']]
X_test  = test.drop('default_ind', axis=1)
y_test  = test.loc[:,['default_ind']]




In [ ]:
 
#OVERSAMPLING-SMOTE
sm= SMOTE(random_state=42)
X_sm, y_sm = sm.fit_sample(X_train, y_train)

# #OVERSAMPLING-RANDOM
# ros= RandomOverSampler(random_state=555)
# X_over, y_over= ros.fit_sample(X_train, y_train)

# #Undersampling
# rus = RandomUnderSampler(return_indices=True, random_state=555)
# X_resampled, y_resampled, idx_resampled= rus.fit_sample(X_train, y_train)

In [ ]:
data_fin.info()
data_fin.describe()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std= scaler.fit_transform(X_sm)
X_std_test= scaler.fit_transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score,classification_report

In [ ]:
#1. Logistic Regression

#SMOTE
lr_sm = LogisticRegression() 
lr_sm.fit(X_sm, y_sm)
lr_sm.score(X_sm, y_sm)
y_pred_sm= lr_sm.predict(X_test)
accuracy_score(y_test, y_pred_sm)
roc_auc_score(y_test, y_pred_sm)
classification_report(y_test, y_pred_sm)
f1_score(y_test, y_pred_sm)

lr_sm.coef_.shape

In [ ]:
#2 Decision Tree
from sklearn.tree import tree, DecisionTreeClassifier
dt = tree.DecisionTreeClassifier(criterion='gini')
dt.fit(X_sm, y_sm)
dt.score(X_sm, y_sm)
y_pred_sm= dt.predict(X_test)
accuracy_score(y_test, y_pred_sm)
roc_auc_score(y_test, y_pred_sm)
classification_report(y_test, y_pred_sm)
f1_score(y_test, y_pred_sm)    

In [ ]:
#3 Random Forests
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators=80, max_features= 'log2')
rf.fit(X_sm, y_sm)
#rf.score(X_sm, y_sm)
y_rf= rf.predict(X_test)
accuracy_score(y_test, y_rf)
roc_auc_score(y_test, y_rf)
classification_report(y_test, y_rf)
f1_score(y_test, y_rf)    

In [ ]:
#tuning RF
from sklearn.grid_search import GridSearchCV
param_grid = { 
    'n_estimators': [20, 80],
    'max_features': [None, 'log2', 'sqrt']
}
CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rf.fit(X_sm, y_sm)
print (CV_rf.best_params)

In [ ]:
#variable importance
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()


In [ ]:
# Gradient Boosted Trees
from sklearn.ensemble import GradientBoostingClassifier  
gb = GradientBoostingClassifier(n_estimators=80, learning_rate=1, 
                                random_state=42)
gb.fit(X_sm, y_sm)
#rf.score(X_sm, y_sm)
y_gb= gb.predict(X_test)
#accuracy_score(y_test, y_rf)
roc_auc_score(y_test, y_gb)
classification_report(y_test, y_gb)
f1_score(y_test, y_gb) 


In [ ]:
# Support Vector Machines
from sklearn import svm
from sklearn.svm import SVC

model_svm = svm.SVC(random_state=42, tol=100,class_weight='balanced')
model_svm.fit(X_std, y_sm)
y_svm= model_svm.predict(X_std_test)
accuracy_score(y_test, y_svm)
roc_auc_score(y_test, y_svm)
classification_report(y_test, y_svm)
f1_score(y_test, y_svm)